Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [2]:
"""This activates packages, namely ambulance deployment"""
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "/Users/Owner/Documents/Austin/simulateUpdate/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON, DelimitedFiles
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

using AmbulanceDeployment, Dates

LoadError: [91mcannot document the following expression:[39m

[91mimport Pkg[39m


In [3]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 2 methods)

In [7]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 30
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 30 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [9]:
"""Simulation Enginer"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*2*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*2*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [10]:
"""call simulation"""
@time df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
calling event id: 2 time: 575 value: 38
Ambulance has arrived on the scene for event 1 at time 638
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
calling event id: 7 time: 1450 value: 76
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 6 at time 1787
Ambulance for event 1 has arrived at the hospital at time 1836
calling event id: 10 time: 1838 value: 67
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1901 value: 82
Ambulance has arrived on the scene for event 4 at time 1912
Ambulance has arrived on the scene for event 2 at time 1924
calling event id: 13 time: 2091 value: 98
calling event id: 14 time: 2117 value: 113
Ambulance has arrived on the scene for event 8 at time 2178
Ambulance has arrived on the scene for event 3 at time 2209


Ambulance has arrived on the scene for event 43 at time 12660
calling event id: 45 time: 12678 value: 99
Ambulance has arrived on the scene for event 44 at time 12679
Ambulance for event 40 has arrived at the hospital at time 12778
Ambulance for event 39 has arrived at the hospital at time 12824
calling event id: 46 time: 12885 value: 124
Ambulance for event 37 has arrived at the hospital at time 12994
Ambulance for event 38 has arrived at the hospital at time 12995
calling event id: 47 time: 13075 value: 144
Ambulance for event 35 is returning to the station at 13085
Ambulance for event 36 is returning to the station at 13115
Ambulance has arrived on the scene for event 45 at time 13278
Ambulance for event 42 has arrived at the hospital at time 13291
Ambulance for event 43 has arrived at the hospital at time 13436
Ambulance for event 41 has arrived at the hospital at time 13478
Ambulance has arrived on the scene for event 46 at time 13485
Ambulance from event 35 has returned to statio

Ambulance for event 75 is returning to the station at 26546
Ambulance for event 74 is returning to the station at 26557
calling event id: 81 time: 26594 value: 114
calling event id: 82 time: 26642 value: 98
calling event id: 83 time: 26657 value: 115
calling event id: 84 time: 26802 value: 98
Ambulance from event 74 has returned to station 17
Ambulance from event 67 has returned to station 2
Ambulance has arrived on the scene for event 80 at time 27323
Ambulance for event 79 has arrived at the hospital at time 27410
Ambulance for event 76 is returning to the station at 27449
calling event id: 85 time: 27495 value: 50
Ambulance for event 78 is returning to the station at 27641
Ambulance from event 73 has returned to station 28
Ambulance for event 77 is returning to the station at 27746
calling event id: 86 time: 27777 value: 82
Ambulance from event 75 has returned to station 1
calling event id: 87 time: 27938 value: 50
Ambulance has arrived on the scene for event 81 at time 27977
Ambula

Ambulance for event 110 is returning to the station at 43588
calling event id: 117 time: 43634 value: 99
calling event id: 118 time: 43834 value: 52
calling event id: 119 time: 43859 value: 113
Ambulance for event 114 has arrived at the hospital at time 44087
Ambulance has arrived on the scene for event 115 at time 44211
Ambulance for event 110 has arrived at the hospital at time 44220
Ambulance for event 114 has arrived at the hospital at time 44255
calling event id: 120 time: 44359 value: 98
Ambulance for event 113 has arrived at the hospital at time 44372
Ambulance from event 107 has returned to station 22
calling event id: 121 time: 44594 value: 113
Ambulance for event 111 is returning to the station at 44603
Ambulance has arrived on the scene for event 116 at time 44689
Ambulance from event 110 has returned to station 14
Ambulance for event 112 is returning to the station at 44948
calling event id: 122 time: 45007 value: 82
Ambulance has arrived on the scene for event 119 at time 

Ambulance for event 132 is returning to the station at 53442
calling event id: 159 time: 53522 value: 98
Ambulance from event 139 has returned to station 8
Ambulance for event 147 has arrived at the hospital at time 53567
calling event id: 160 time: 53646 value: 98
160: call from 98 is a shortfall
Ambulance has arrived on the scene for event 155 at time 53766
calling event id: 161 time: 53770 value: 82
Ambulance for event 152 has arrived at the hospital at time 53775
Ambulance for event 137 is returning to the station at 53818
Ambulance has arrived on the scene for event 157 at time 53832
Ambulance for event 149 has arrived at the hospital at time 53843
Ambulance for event 150 has arrived at the hospital at time 53845
calling event id: 162 time: 53901 value: 98
162: call from 98 is a shortfall
calling event id: 163 time: 53913 value: 51
calling event id: 164 time: 53914 value: 66
Ambulance has arrived on the scene for event 154 at time 54040
Ambulance from event 135 has returned to sta

...

Ambulance has arrived on the scene for event 918 at time 258332
Ambulance from event 904 has returned to station 28
calling event id: 920 time: 258391 value: 68
920: call from 68 is a shortfall
Ambulance from event 898 has returned to station 25
calling event id: 920 time: 258406 value: 68
920: call from 68 is a shortfall
calling event id: 921 time: 258428 value: 142
Ambulance for event 903 is returning to the station at 258497
Ambulance for event 912 is returning to the station at 258563
Ambulance for event 917 has arrived at the hospital at time 258590
calling event id: 922 time: 258634 value: 69
922: call from 69 is a shortfall
Ambulance for event 901 has arrived at the hospital at time 258652
Ambulance from event 891 has returned to station 44
calling event id: 920 time: 258717 value: 68
920: call from 68 is a shortfall
calling event id: 922 time: 258717 value: 69
This is a shortfallcalling event id: 923 time: 258844 value: 83
calling event id: 924 time: 258868 value: 142
Ambulance

Ambulance has arrived on the scene for event 939 at time 263361
Ambulance for event 930 is returning to the station at 263404
calling event id: 950 time: 263413 value: 128
950: call from 128 is a shortfall
Ambulance for event 920 is returning to the station at 263425
Ambulance for event 931 has arrived at the hospital at time 263513
Ambulance for event 922 is returning to the station at 263513
Ambulance has arrived on the scene for event 944 at time 263519
Ambulance for event 932 is returning to the station at 263545
Ambulance from event 925 has returned to station 17
calling event id: 934 time: 263561 value: 114
934: call from 114 is a shortfall
calling event id: 950 time: 263561 value: 128
This is a shortfallAmbulance has arrived on the scene for event 941 at time 263568
Ambulance from event 920 has returned to station 8
calling event id: 934 time: 263729 value: 114
934: call from 114 is a shortfall
calling event id: 951 time: 263745 value: 52
calling event id: 952 time: 263776 value

calling event id: 957 time: 267598 value: 40
957: call from 40 is a shortfall
calling event id: 967 time: 267598 value: 115
967: call from 115 is a shortfall
Ambulance from event 931 has returned to station 43
calling event id: 957 time: 267605 value: 40
This is a shortfallcalling event id: 967 time: 267605 value: 115
967: call from 115 is a shortfall
Ambulance from event 950 has returned to station 17
calling event id: 967 time: 267719 value: 115
967: call from 115 is a shortfall
Ambulance has arrived on the scene for event 962 at time 267777
Ambulance for event 951 is returning to the station at 267784
Ambulance for event 960 has arrived at the hospital at time 267793
Ambulance from event 933 has returned to station 4
calling event id: 967 time: 267970 value: 115
This is a shortfallAmbulance for event 953 is returning to the station at 268003
Ambulance from event 951 has returned to station 8
Ambulance has arrived on the scene for event 966 at time 268106
Ambulance has arrived on the

Ambulance for event 987 is returning to the station at 280156
Ambulance has arrived on the scene for event 992 at time 280218
Ambulance from event 981 has returned to station 31
Ambulance from event 984 has returned to station 1
Ambulance has arrived on the scene for event 993 at time 280347
Ambulance for event 989 is returning to the station at 280576
Ambulance from event 988 has returned to station 15
Ambulance for event 991 has arrived at the hospital at time 280763
Ambulance has arrived on the scene for event 995 at time 280869
Ambulance for event 993 has arrived at the hospital at time 281001
Ambulance has arrived on the scene for event 994 at time 281090
Ambulance for event 992 has arrived at the hospital at time 281269
calling event id: 996 time: 281271 value: 113
Ambulance from event 989 has returned to station 25
Ambulance from event 987 has returned to station 3
calling event id: 997 time: 281699 value: 99
Ambulance for event 995 has arrived at the hospital at time 281719
Amb

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 10.0         │ 19.9667   │
│ 2    │ 2     │ 43            │ 0.0      │ 22.4833      │ 11.75     │
│ 3    │ 3     │ 14            │ 0.0      │ 24.45        │ 19.95     │
│ 4    │ 4     │ 34            │ 0.0      │ 20.35        │ 13.2833   │
│ 5    │ 5     │ 17            │ 0.0      │ 19.5333      │ 18.0667   │
│ 6    │ 6     │ 22            │ 0.0      │ 10.0         │ 11.75     │
│ 7    │ 7     │ 39            │ 0.0      │ 49.0833      │ 26.8333   │
│ 8    │ 8     │ 8             │ 0.0      │ 10.0         │ 29.8833   │
│ 9    │ 9     │ 8             │ 0.0      │ 22.7         │ 17.9      │
│ 10   │ 10    │ 3             │ 0.0      │ 19.2333      │ 14.2      │
⋮
│ 990  │ 990   │ 44      

In [4]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

LoadError: [91mUndefVarError: guiArray not defined[39m

In [73]:
mean(df["responsetime"])

62.19070000000001

In [74]:
using DelimitedFiles
filename = string(model_name,string(namb),"calls",string(ncalls),".csv")
writedlm(filename,  df["responsetime"])
print(filename)

Stochastic30calls1000.csv

"40"